In [1]:
# location configuration for saving the data
try:
    configs = {"fs.azure.account.auth.type": "OAuth",
               "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
               "fs.azure.account.oauth2.client.id": "1af30219-3654-4f03-95df-ed5c5afe85bf",
               "fs.azure.account.oauth2.client.secret": "Z:]B8fC3CHpwQ9fCH4QpVQur:f?46/Pr",
               "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/b7f604a0-00a9-4188-9248-42f3a5aac2e9/oauth2/token"}
    try:
        dbutils.fs.unmount('/mnt/shrink_data')
        print('Filesystem unmounted')        
    except:
      pass
    finally:
        dbutils.fs.mount(
          source = "abfss://absitdsdevwussa001@absitdsdevwussa001.dfs.core.windows.net/default/enriched/product/unrestricted/shrink_data",
          mount_point = "/mnt/shrink_data",
          extra_configs = configs)
        print('Filesystem mounted')
except:
    pass

/mnt/shrink_data has been unmounted.
Filesystem unmounted
Filesystem mounted

In [2]:

from pyspark.sql import SQLContext

from pyspark.sql import HiveContext
hiveCtx = HiveContext(sc)
import sqlite3 as sq
import pandas as pd
import numpy as np
import time
import re
from pandas import ExcelWriter

In [3]:
%sql
select * from db_enriched.bdr_purchase_item limit 10;

corporation_id,division_id,vend_nbr,vend_sub_acct_nbr,bdr_vend_sub_nbr,bdr_invc_nbr,upc_id,recv_dt,department_id,invc_cd,load_dt,bdr_vend_cst,fin_rtl_prc_amt,off_invc_amt,invc_void_ind,recv_qty,pack_qty,cash_disc_pct,terms_id,dep_amt,itm_recv_uom_cd,unauth_item_ind,base_cst,store_cst,store_pack_qty,batch_id,purchase_order_nbr,purchase_order_dt,feed_cd,last_update_ts,reg_rtl_prc_fctr,batchid,store_id
1,32,015099,001,901,189011344024,7371141551,2018-12-10,336,R,2018-12-11,2.77,3.99,0.8,0,1.0,1,0.0,1,0.0,EA,0,2.77,2.77,1,8015,,0001-01-01,BPI,2018-12-11T01:32:06.000+0000,1,20190612022728,1
1,32,029413,001,701,112340,60888300005,2018-11-30,301,D,2018-12-01,15.0,1.99,0.0,0,1.0,12,0.0,2,0.0,CA,0,15.0,15.0,12,8005,,0001-01-01,BPI,2018-12-01T03:27:16.000+0000,1,20190612022728,1
1,32,001179,024,924,2431673,8520091874,2018-12-10,303,D,2018-12-11,27.25,6.99,0.0,0,1.0,6,0.0,1,0.05,CA,0,27.25,27.25,6,8015,,0001-01-01,BPI,2018-12-11T01:32:06.000+0000,1,20190612022728,1
1,32,003607,001,801,52237714383,4812125505,2018-11-30,336,D,2018-12-01,3.55,4.69,0.95,0,3.0,1,0.0,2,0.0,EA,0,3.55,3.55,1,8005,,0001-01-01,BPI,2018-12-01T03:27:16.000+0000,1,20190612022728,1
1,32,014563,001,701,IN3343685,3022302319,2018-12-03,306,D,2018-12-04,3.85,6.99,0.0,0,3.0,1,0.0,2,0.0,EA,0,3.85,3.85,1,8008,714065,2018-12-01,BPI,2018-12-04T02:42:22.000+0000,1,20190612022728,1
1,32,003607,001,801,52237714383,7203000237,2018-11-30,336,D,2018-12-01,5.31,6.99,2.41,0,4.0,1,0.0,2,0.0,EA,0,5.31,5.31,1,8005,,0001-01-01,BPI,2018-12-01T03:27:16.000+0000,1,20190612022728,1
1,32,014563,001,701,IN3343143,84637909949,2018-12-01,306,D,2018-12-02,3.39,6.0,0.0,0,2.0,1,0.0,2,0.0,EA,0,3.39,3.39,1,8006,713807,2018-11-29,BPI,2018-12-02T02:21:41.000+0000,1,20190612022728,1
1,32,006446,004,904,4862386,85418300627,2018-12-11,301,D,2018-12-12,18.06,4.49,0.0,0,1.0,6,0.0,1,0.0,CA,0,18.06,18.06,6,8016,384758,2018-12-10,BPI,2018-12-12T01:33:04.000+0000,1,20190612022728,1
1,32,029413,001,701,112340,65722700050,2018-11-30,301,D,2018-12-01,19.0,2.49,0.0,0,2.0,12,0.0,2,0.0,CA,0,19.0,19.0,12,8005,,0001-01-01,BPI,2018-12-01T03:27:16.000+0000,1,20190612022728,1
1,32,001179,024,924,2431673,8105499476,2018-12-10,303,D,2018-12-11,81.25,8.99,0.0,0,1.0,12,0.0,1,0.05,CA,0,81.25,81.25,12,8015,,0001-01-01,BPI,2018-12-11T01:32:06.000+0000,1,20190612022728,1


In [4]:
%sql
select COUNT(*) from (
select  * from db_enriched.bdr_purchase_item where recv_dt between ("2018-01-01") and ("2018-12-31"));

--select count(*), store_id from  db_enriched.bdr_purchase_item WHERE recv_dt between ("2018-01-01") and ("2018-12-31")  group by department_id;

count(1)
227172861


In [5]:
%sql
select count(*),itm_recv_uom_cd from db_enriched.bdr_purchase_item group by itm_recv_uom_cd  order by itm_recv_uom_cd  desc;
select count(*),store_id  from db_enriched.bdr_purchase_item group by store_id  order by store_id  desc;

count(1),itm_recv_uom_cd
251411145,EA
304588217,CA
2929838,#
1250232,


In [7]:
%sql
select count(*),store_id  from db_enriched.bdr_purchase_item group by store_id  order by store_id  desc;

count(1),store_id
336,9887
22,9878
1,9855
654,9123
1875,5804
3066,5802
6906,5801
3654,5800
155456,4933
400825,4932


In [8]:
%sql

create table db_work.atest
select  * from db_enriched.bdr_purchase_item where recv_dt between ("2018-01-01") and ("2018-12-31")  AND store_id IN (3529,3557);



In [9]:
%sql
-- additional items 70%

create table db_work.a4a as 
select  * from db_enriched.bdr_purchase_item where recv_dt between ("2018-01-01") and ("2018-12-31")  
AND store_id IN  (4904,4513,4469,4405,4404,4387,4381,4333,4318,4316,4313,4288,4262,3595,3593,3564,3557,3553,3542,3541,3531,3529,
3508,3414,3212,3211,3136,3134,2993,2790,2723,2696,2690,2637,2631,2627,2623,2604,2599,2592,2528,2448);



In [10]:
#df=hiveCtx.sql("select * from db_work.a4a")
df.toPandas().to_csv ('/dbfs/mnt/shrink_data/bdr_purchase_item.csv',header='True', encoding="utf-8")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1357899953788288> in <module> () 
 1 #df=hiveCtx.sql("select * from db_work.a4a") 
 ----> 2 df . toPandas ( ) . to_csv ( '/dbfs/mnt/shrink_data/bdr_purchase_item.csv' , header = 'True' , encoding = "utf-8" ) 

 NameError : name 'df' is not defined

In [11]:
%sql
--drop table db_work.bdr2018;
create table db_work.bdr2018 as 
select  * from db_enriched.bdr_purchase_item where recv_dt between ("2018-01-01") and ("2018-3-31");

In [12]:
train = sqlContext.table("db_work.bdr")
#train_pdf = train.select("*").toPandas()
#train_pdf.head(10)

In [13]:
%sql
select COUNT(*) from (db_work.bdr)

count(1)
227172861


In [14]:
%sql
select min(recv_dt ),max(recv_dt ) from db_enriched.bdr_purchase_item

min(recv_dt),max(recv_dt)
1953-03-14,2048-05-27


In [15]:
%sql
----select COUNT(store_id) from (db_work.bdr2018)
--WHERE upc_id IS NULL
--CREATE TABLE  Agg_bdr2018AGG
--INSERT INTO db_work.a4a


SELECT store_id
,mean (store_cst )as Avg_store_cst
,mean (dep_amt )as Avg_dep_amt
,mean (bdr_vend_cst) as Avg_bdr_vend_cst
,mean (fin_rtl_prc_amt)as Avg_fin_rtl_prc_amt
,mean (recv_qty)as Avg_recv_qty
,mean (pack_qty)as Avg_pack_qty
,mean (cash_disc_pct) as Avg_cash_disc_pct
--mean (dep_amt)as Avg_dep_amt,
,mean (base_cst) as Avg_base_cst
,mean (store_cst)as Avg_store_cst
,mean (store_pack_qty)as Avg_store_pack_qty
,mean (reg_rtl_prc_fctr)as Avg_reg_rtl_prc_fctr
 FROM db_work.bdr2018
GROUP BY store_id;


store_id,Avg_store_cst,Avg_dep_amt,Avg_bdr_vend_cst,Avg_fin_rtl_prc_amt,Avg_recv_qty,Avg_pack_qty,Avg_cash_disc_pct,Avg_base_cst,Avg_store_cst,Avg_store_pack_qty,Avg_reg_rtl_prc_fctr
3997,18.37198057,0.0,18.6757572,5.01889213,3.88259043,11.5334,0.014888,18.95473096,18.37198057,11.5323,1.1165
833,19.41082256,0.0,19.93453721,5.22847445,4.33687951,12.0006,0.014403,20.24811413,19.41082256,11.9986,1.0957
471,17.9089914,0.0,18.0666069,7.40519924,4.86463772,5.8039,0.01683,19.46278228,17.9089914,5.8026,1.0051
1645,16.77578797,0.0,16.93061933,6.93474713,5.21333546,5.5977,0.031033,18.2941864,16.77578797,5.5972,1.0059
2142,17.84047327,0.15153107,17.90139713,8.08300203,4.91947164,5.0738,0.096654,19.27910833,17.84047327,5.0658,1.0001
496,17.48316492,0.0,17.62341867,7.18945267,4.24446253,5.5879,0.030511,18.56669574,17.48316492,5.5906,1.0035
1580,16.14791122,0.26589206,16.19351227,7.0905671,5.17847587,5.0649,0.041421,17.25034871,16.14791122,5.0645,1.0069
2659,16.72235063,0.15358928,16.81685189,7.55593228,5.06605788,4.9722,0.114554,18.40477948,16.72235063,4.9612,1.0003
1591,12.90342775,0.0,13.03148734,6.47690582,8.37969129,4.3966,0.046119,14.51033874,12.90342775,4.3902,1.0777
3175,15.50156012,0.13825537,15.52485411,7.30519108,5.20513929,4.6621,0.108005,16.91790939,15.50156012,4.6584,1.0001


In [16]:
bdr_purchase_item

In [17]:
%sql
select SUM(*) from (db_work.bdr2018)
WHERE store_id IS NULL

count(1)
0


In [18]:
%sql
select COUNT(*) from (db_work.bdr2018)
WHERE purchase_order_dt IS NULL


count(1)
0


In [19]:
%sql
--drop table db_work.new_bdr2018;
create table db_work.working_bdr2018 as
select  * from db_enriched.bdr_purchase_item where purchase_order_dt  >= date("2017-07-01") and recv_dt <("2019-01-01");

In [21]:
%sql
--drop table db_work.final_bdr2018;
create table db_work.final_bdr2018 as  SELECT *
    FROM (
        SELECT store_id,vend_nbr,recv_dt
              
            
               ,SUM (vend_sub_acct_nbr) as Agg_vend_sub_acct_nbr
              ,SUM (store_cst)as Agg_store_cst
              ,sum (bdr_vend_cst)as Agg_bdr_vend_cst
               ,SUM (fin_rtl_prc_amt)as Agg_fin_rtl_prc_amt
               ,SUM (off_invc_amt)as Agg_off_invc_amt
               ,SUM (pack_qty)as Agg_pack_qty
               ,sum(cash_disc_pct)as Agg_cash_disc_pct             
               ,SUM (base_cst)as Agg_base_cst
               ,SUM (store_pack_qty)as Agg_store_pack_qty
              ,SUM (reg_rtl_prc_fctr)as Agg_reg_rtl_prc_fctr
              ,SUM (dep_amt)as Agg_dep_amt
               ,SUM (recv_qty)as Agg_recv_qty             

           ,RANK() OVER (
                  PARTITION BY store_id
                  ORDER BY count(vend_nbr) DESC
              ) AS rank
        FROM (
            SELECT *
            FROM db_work.working_bdr2018
            WHERE store_id IN (
                SELECT store_id
                FROM db_work.working_bdr2018
                GROUP BY store_id 
                ORDER BY count(vend_nbr) DESC
               -- LIMIT 5
            )
        )
       GROUP BY  store_id,vend_nbr,recv_dt
        --vend_nbr, bdr_vend_sub_nbr,department_id,division_id,upc_id
    )
    

In [22]:
%sql
select COUNT(*) from (
select  * from db_work.final_bdr2018 )

count(1)
851009


In [23]:
%sql
create table db_work.afinal_bdr2018 as  SELECT *
    FROM (
        SELECT store_id
                store_id,department_id,division_id,vend_nbr,recv_dt
               
               ,SUM (Agg_vend_sub_acct_nbr) as Agg_vend_sub_acct_nbr
              ,SUM (Agg_store_cst)as Agg_store_cst
              ,sum (Agg_bdr_vend_cst)as Agg_bdr_vend_cst
               ,SUM (Agg_fin_rtl_prc_amt)as Agg_fin_rtl_prc_amt
               ,SUM (Agg_off_invc_amt)as Agg_off_invc_amt
               ,SUM (Agg_pack_qty)as Agg_pack_qty
               ,sum(Agg_cash_disc_pct)as Agg_cash_disc_pct             
               ,SUM (Agg_base_cst)as Agg_base_cst
               ,SUM (Agg_store_pack_qty)as Agg_store_pack_qty
              ,SUM (Agg_reg_rtl_prc_fctr)as Agg_reg_rtl_prc_fctr
              ,SUM (Agg_dep_amt)as Agg_dep_amt
                            
             ,RANK() OVER (
                  PARTITION BY store_id
                  ORDER BY count(department_id) DESC
              ) AS rank
        FROM (
            SELECT *
            FROM db_work.a_bdr2018
            WHERE store_id IN (
                SELECT store_id
                FROM db_work.a_bdr2018
                GROUP BY store_id
                ORDER BY count(department_id) DESC
               -- LIMIT 5
            )
        )
       GROUP BY  store_id,department_id,division_id,vend_nbr,recv_dt
        --vend_nbr, bdr_vend_sub_nbr,department_id,division_id,upc_id
    )

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table db_work.afinal_bdr2018 already exists. You need to drop it first.;
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:158)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:110)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:121)
	at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:205)
	at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:205)
	at org.apache.spark.sql.Dataset$$anonfun$56.apply(Dataset.scala:3438)
	at org.apache.spark.sql.Dataset$$anonfun$56.apply(Dataset.scala:3433)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$withAction(Dataset.scala:3433)
	at org.apache.spark.sql.Dataset. (Dataset.scala:205)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:89)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:696)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:716)
	at com.databricks.backend.daemon.driver.SQLDriverLocal$$anonfun$1.apply(SQLDriverLocal.scala:88)
	at com.databricks.backend.daemon.driver.SQLDriverLocal$$anonfun$1.apply(SQLDriverLocal.scala:34)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutable.List.map(List.scala:296)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:34)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:141)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$8.apply(DriverLocal.scala:368)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$8.apply(DriverLocal.scala:345)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:271)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:345)
	at com.databricks.backend.daemon.driver.DriverWrapper$$anonfun$tryExecutingCommand$2.apply(DriverWrapper.scala:644)
	at com.databricks.backend.daemon.driver.DriverWrapper$$anonfun$tryExecutingCommand$2.apply(DriverWrapper.scala:644)
	at scala.util.Try$.apply(Try.scala:192)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:639)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:485)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:597)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:390)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
	at com.databricks.backend.daemon.drive

In [24]:
df=hiveCtx.sql("select * from db_work.final_bdr2018 ")
df.toPandas().to_csv ('/dbfs/mnt/shrink_data/Final_bdr_purchase_item.csv',header='True', encoding="utf-8")

In [25]:
%sql
select count(*),bdr_vend_sub_nbr from db_work.bdr2018 group by bdr_vend_sub_nbr order by bdr_vend_sub_nbr  desc;


count(1),bdr_vend_sub_nbr
2307,999
761,998
375,997
903,950
1283008,936
194664,913
12,909
166251,907
362253,906
5871316,905


In [26]:
%sql
Drop table db_work.final_bdr2018;
create table db_work.final_bdr2018 as  SELECT *
    FROM (
        SELECT store_id,
        recv_dt,
        purchase_order_dt


,count(bdr_vend_cst)as bdr_vend_cst
,AVG(fin_rtl_prc_amt)as fin_rtl_prc_amt
,AVG(off_invc_amt)as off_invc_amt
,AVG(invc_void_ind)as invc_void_ind
,AVG(recv_qty)as recv_qty
,AVG(pack_qty)as pack_qty
,AVG(cash_disc_pct)as cash_disc_pct
,AVG(terms_id)as terms_id
,AVG(dep_amt)as dep_amt
,AVG(itm_recv_uom_cd)as itm_recv_uom_cd
,AVG(unauth_item_ind)as unauth_item_ind
,AVG(base_cst)as base_cst
,AVG(store_cst)as store_cst
,AVG(store_pack_qty)as store_pack_qty
,AVG(reg_rtl_prc_fctr)as reg_rtl_prc_fctr
,count(purchase_order_nbr)as purchase_order_nbr

from db_work.working_bdr2018

Group By store_id,recv_dt,purchase_order_dt ORDER BY recv_dt);
              

In [27]:
%sql
select COUNT(*) from (
select  * from db_work.final_bdr2018 )

count(1)
544231


In [28]:
%sql
select * from db_work.final_bdr2018 limit 10;

store_id,recv_dt,bdr_vend_cst,fin_rtl_prc_amt,off_invc_amt,invc_void_ind,recv_qty,pack_qty,cash_disc_pct,terms_id,dep_amt,itm_recv_uom_cd,unauth_item_ind,base_cst,store_cst,store_pack_qty,reg_rtl_prc_fctr,purchase_order_nbr
677,2018-12-05,130,971.56,17.94,0.0,160.0,899,0.0,136,0.0,null,0.0,2744.5064,2760.7664,899,158,130
1109,2018-12-05,3,33.98,0.0,0.0,540.0,3,0.0,3,2.8,null,0.0,16.06,16.06,3,5,3
127,2018-12-05,25,113.28,31.74,0.0,25.0,206,0.0,75,0.0,null,0.0,616.32,616.32,206,27,25
578,2018-12-05,91,450.53,35.34,0.0,99.0,860,0.0,92,0.0,null,0.0,2073.4878,2013.6078,860,97,91
1329,2018-12-05,193,1494.33,94.74,0.0,296.0,1570,0.0,193,0.0,null,0.0,4295.916,4295.895,1570,193,193
1621,2018-12-05,15,87.75,0.0,0.0,60.0,219,0.0,15,0.0,null,0.0,498.2471,498.2451,219,15,15
3208,2018-12-05,14,85.56,0.0,0.0,60.0,188,0.0,14,0.0,null,0.0,451.1411,451.8906,188,14,14
2576,2018-12-05,393,1666.99,543.43,0.0,541.0,6679,0.0,395,0.0,null,0.0,9885.17,9822.9,6679,411,393
3424,2018-12-05,197,1062.53,48.09,0.0,213.0,1635,0.0,197,0.0,null,0.0,5152.77,5152.77,1635,197,197
1596,2018-12-05,9,63.41,0.0,0.0,28.0,9,0.0,9,0.0,null,0.0,38.01,38.01,9,9,9
